Code used to create result file for tying out decision variable, objective function and constraint code.

In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
#get the distinct list of city and demand type combinations

list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['paris_db=1.00', 'berlin_db=1.00', 'berlin_db=0.50', 'frankfurt_db=2.00', 'lyon_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'paris_db=2.00', 'lyon_db=1.00', 'paris_db=0.50', 'frankfurt_db=1.00', 'paris_db=4.00', 'lyon_db=0.50', 'berlin_db=2.00', 'frankfurt_db=4.00', 'lyon_db=4.00']
16


In [3]:
#create a dataframe that can hold all result information

list_results = []

workforce_dict = {0:5, 1:5, 2:5, 3:5, 4:5}
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in list_city_dt:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"
    
    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.5]:
        model = 'fixed'
        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        df_ = pd.DataFrame(dict_results)
        list_results.append(df_)

        model = 'flex'
        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
        df_ = pd.DataFrame(dict_results)
        list_results.append(df_)

        model = 'partflex'
        for max_n_shifts in range(2,5):            
            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
            df_ = pd.DataFrame(dict_results)
            list_results.append(df_)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35780 rows, 39520 columns and 208958 nonzeros
Model fingerprint: 0xb4b9c01f
Variable types: 33600 continuous, 5920 integer (5920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 35105 rows and 37352 columns
Presolve time: 0.17s
Presolved: 675 rows, 2168 columns, 6183 nonzeros
Variable types: 753 continuous, 1415 integer (1400 binary)
Found heuristic solution: objective 16852.894682

Root relaxat

In [4]:
df_results = pd.concat(list_results, ignore_index = True)
df_results.drop(columns = ['wage_costs','objective_value_post_wage'], inplace = True)
df_results.sort_values(by = ['city','demand_baseline','model','outsourcing_cost_multiplier'], inplace = True)

df_results.to_csv(r'../rostering_results/results_cm.csv', index = False)

In [6]:
df_results = pd.concat(list_results, ignore_index = True)
df_results.drop(columns = ['wage_costs','objective_value_post_wage'], inplace = True)
df_results.sort_values(by = ['city','demand_baseline','model','outsourcing_cost_multiplier'], inplace = True)

df_results.to_excel(r'../rostering_results/results_cm.xlsx', index = False)

- Import results back in and tie out

In [5]:
df_cm = pd.read_csv(r'../rostering_results/results_cm.csv')
df_cm.rename(columns = {'outsourcing_cost_multiplier':'OC'}, inplace = True)
df_cm = df_cm[['city','model','demand_baseline','OC','objective_value']]
df_cm.head()

,city,model,demand_baseline,OC,objective_value
0,berlin,fixed,0.5,1.2,3276.60
1,berlin,fixed,0.5,1.5,3975.75
2,berlin,fixed,0.5,1.8,4674.90
3,berlin,fixed,0.5,2.0,5141.00
4,berlin,fixed,0.5,2.5,6306.25


In [6]:
list_jonny = []

for file_in in os.listdir(r'../rostering_results'):
    if 'jonny' in file_in:
        # print(file_in)
        df_ = pd.read_csv(f'../rostering_results/{file_in}')
        split_ = file_in.split('_')
        df_['city'] = split_[0]
        df_['demand_baseline'] = pd.to_numeric(split_[1].split('=')[1])
        df_['OC'] = pd.to_numeric(split_[4].split('=')[1])
        df_ = df_[['city','model','shift_type','demand_baseline','OC','obj_value']]
        list_jonny.append(df_)

df_jonny = pd.concat(list_jonny, ignore_index = True)
df_jonny['shift_type'] = df_jonny['shift_type'].str.replace('3','').str.strip()
df_jonny = df_jonny[df_jonny['model']=='roster']
df_jonny.drop(columns = ['model'], inplace = True)
df_jonny.rename(columns = {'shift_type':'model'}, inplace = True)
df_jonny.sort_values(by = ['city','model'], inplace = True)
df_jonny.head()

,city,model,demand_baseline,OC,obj_value
13,berlin,fixed,1.0,1.5,8370.116667
3,berlin,partflex,1.0,1.5,9381.550000
11,frankfurt,fixed,1.0,1.5,4224.500000
7,frankfurt,partflex,1.0,1.5,4943.000000
1,lyon,fixed,1.0,1.5,7032.120437


In [7]:
#compare

df_compare = df_jonny.merge(df_cm, how = 'left', on = ['city','model','demand_baseline','OC'])
df_compare['diff'] = (df_compare['obj_value']-df_compare['objective_value']).abs().round(4)
df_compare

,city,model,demand_baseline,OC,obj_value,objective_value,diff
0,berlin,fixed,1.0,1.5,8370.116667,8370.116667,0.0
1,berlin,partflex,1.0,1.5,9381.550000,NaN,NaN
2,berlin,partflex,1.0,1.5,9381.550000,NaN,NaN
3,berlin,partflex,1.0,1.5,9381.550000,NaN,NaN
4,frankfurt,fixed,1.0,1.5,4224.500000,4224.500000,0.0
5,frankfurt,partflex,1.0,1.5,4943.000000,NaN,NaN
6,frankfurt,partflex,1.0,1.5,4943.000000,NaN,NaN
7,frankfurt,partflex,1.0,1.5,4943.000000,NaN,NaN
8,lyon,fixed,1.0,1.5,7032.120437,7032.120437,0.0
9,lyon,partflex,1.0,1.5,7585.464980,NaN,NaN
